In [7]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn
import numpy as np
import plotly.graph_objects as go

In [8]:
data = pd.read_csv('data/TOTAL.csv', delimiter=';')
data.head()

,Players,Ratings,Minutes,Goals,Assists,Offsides,Passes,Accurate_Passes,Accurate_Passes_%,Crosses,Tackles,Total_Shots,Shots_on_Target,Blocked_Shots,Yellow_Cards,Red_Cards,Fouls_Drawn,Fouls_Committed,Clearances,Saves
0,D. Abramowicz,"7,53","90,00","1,00","0,00","2,00","74,00","57,00","77,03","7,00","1,00","4,00","1,00","0,00","0,00","0,00","2,00","1,00","23,00","0,00"
1,S. Kerk,"6,91","76,33","0,00","1,00","0,00","71,00","52,00","73,24","17,00","2,00","6,00","2,00","2,00","0,00","0,00","2,00","2,00","5,00","0,00"
2,J. Oliveira,"6,90","44,33","0,00","0,00","2,00","30,00","24,00","80,00","4,00","1,00","3,00","2,00","0,00","0,00","0,00","4,00","0,00","0,00","0,00"
3,A. Nguiamba,"4,49","29,67","0,00","0,00","0,00","33,00","28,00","84,85","0,00","4,00","0,00","0,00","0,00","0,00","0,00","1,00","1,00","2,00","0,00"
4,D. Zator,"4,53","22,00","0,00","0,00","0,00","37,00","28,00","75,68","3,00","4,00","0,00","0,00","0,00","0,00","0,00","1,00","0,00","3,00","0,00"


In [9]:
data = data.set_index('Players')
data.head()

,Ratings,Minutes,Goals,Assists,Offsides,Passes,Accurate_Passes,Accurate_Passes_%,Crosses,Tackles,Total_Shots,Shots_on_Target,Blocked_Shots,Yellow_Cards,Red_Cards,Fouls_Drawn,Fouls_Committed,Clearances,Saves
Players,,,,,,,,,,,,,,,,,,,
D. Abramowicz,"7,53","90,00","1,00","0,00","2,00","74,00","57,00","77,03","7,00","1,00","4,00","1,00","0,00","0,00","0,00","2,00","1,00","23,00","0,00"
S. Kerk,"6,91","76,33","0,00","1,00","0,00","71,00","52,00","73,24","17,00","2,00","6,00","2,00","2,00","0,00","0,00","2,00","2,00","5,00","0,00"
J. Oliveira,"6,90","44,33","0,00","0,00","2,00","30,00","24,00","80,00","4,00","1,00","3,00","2,00","0,00","0,00","0,00","4,00","0,00","0,00","0,00"
A. Nguiamba,"4,49","29,67","0,00","0,00","0,00","33,00","28,00","84,85","0,00","4,00","0,00","0,00","0,00","0,00","0,00","1,00","1,00","2,00","0,00"
D. Zator,"4,53","22,00","0,00","0,00","0,00","37,00","28,00","75,68","3,00","4,00","0,00","0,00","0,00","0,00","0,00","1,00","0,00","3,00","0,00"


In [10]:
data = data.replace(',', '.', regex=True)
data = data.apply(pd.to_numeric, errors='coerce')

In [11]:
# Create the figure
from torch import layout


fig = go.Figure()

# List of metrics to plot
metrics = ['Ratings', 'Minutes', 'Goals', 'Assists', 'Offsides', 'Passes',
       'Accurate_Passes', 'Accurate_Passes_%', 'Crosses', 'Tackles',
       'Total_Shots', 'Shots_on_Target', 'Blocked_Shots', 'Yellow_Cards',
       'Red_Cards', 'Fouls_Drawn', 'Fouls_Committed', 'Clearances', 'Saves']

# Add traces for each metric
for metric in metrics:
    filtered_data = data[data[metrics] >  0]
    sorted_df = filtered_data.sort_values(by=metric, ascending=False)
    
    fig.add_trace(go.Bar(x=sorted_df.index, y=sorted_df[metric], name=metric))


# Define visibility matrix
n = len(metrics)
visibility_matrix = {
    "None": [False] * n,
    "AVG Rating": [metric in ['Ratings'] for metric in metrics],
    "Minutes": [metric in ['Minutes'] for metric in metrics],
    "Passes": [metric in ['Passes', 'Accurate_Passes', 'Accurate_Passes_%', 'Crosses'] for metric in metrics],
    "Attack": [metric in ['Goals', 'Assists', 'Total_Shots', 'Shots_on_Target', 'Shots_Inside_Box', 'Shots_Outside_Box'] for metric in metrics],
    "Defense": [metric in ['Tackles', 'Fouls_Drawn', 'Fouls_Committed', 'Saves'] for metric in metrics],
    "Cards": [metric in ['Yellow_Cards', 'Red_Cards'] for metric in metrics],
    "All": [True] * n
}

# Create buttons
buttons = [
    dict(label=label,
         method="update",
         args=[{"visible": visibility},
               {"title": f"Arka Match Stats: {label}"}])
    for label, visibility in visibility_matrix.items()
]

# Add dropdown to figure
fig.update_layout(
    updatemenus=[dict(
        active=0,
        buttons=buttons,
        x=1.1,
        y=1.15
    )],
    title="Arka Match Stats",
    xaxis_title="Match",
    yaxis_title="Value",
    barmode='group'
)

fig.show()
fig.write_html("arka_stats_players.html")